# pandapower-Netze modellieren

Dieses Tutorial enthält eine Einführung in die pandapower-Datenstrukturen und zeigt, wie man mit Hilfe der pandapower-API ein Netzwerk modelliert. Das folgende Beispiel demonstriert den Einsatz der gängigsten Komponenten, die in pandapower verfügbar sind.

<img src="pics/example_network_simple.png">

Die Datenstruktur der pandapower-Umgebung basiert auf der Python-Bibliothek pandas. Ein pandapower-Netzwerk besteht aus einer separaten Komponententabelle für jeden Komponententyp, der im Netzwerk eingesetzt wird. Tabellenspalten repräsentieren die Parameter der jeweiligen Komponente. 

Durch das Ausführen der folgenden Code-Zellen generieren Sie verschiedene Komponententabellen. Weitere Informationen zu den verwendeten Komponenten finden Sie in der pandapower-Dokumentation im Abschnitt "Datastructures and Elements".


### Leeres Netzwerk

Zunächst wird das pandapower-Modul importiert und anschließend ein leerer Netzcontainer erzeugt:

In [1]:
import pandapower as pp #import pandapower

net = pp.create_empty_network() #create an empty network

### Buses

<img src="pics/example_network_simple_buses.png">

Nun werden die Netzknoten generiert, die in pandapower als "buses" bezeichnet werden. Weitere Eingabeparameter sind die Nennspannungen an den Knoten (vn_kv), die gleichzeitig den Startwert für den zur Berechnung eingesetzten Newton-Raphson-Algorithmus darstellen. Ein Name kann zur besseren Lesbarkeit angegeben werden.

Insgesamt werden drei Hochspannungsknoten (vn_kv=110.) und vier Mittelspannungsknoten (vn_kv=20.) benötigt. 

In [2]:
bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = pp.create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = pp.create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = pp.create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = pp.create_bus(net, name="MV Main Bus", vn_kv=20, type="b")
bus6 = pp.create_bus(net, name="MV Bus 1", vn_kv=20, type="b")
bus7 = pp.create_bus(net, name="MV Bus 2", vn_kv=20, type="b")

Bus 3 und bus 4 wurden als "nodes" klassifiziert (type="n"), alle anderen werden als "busbars" eingestuft (type="b"):

In [3]:
net.bus # show bus table

,name,vn_kv,type,zone,in_service
0,HV Busbar,110.0,b,None,True
1,HV Busbar 2,110.0,b,None,True
2,HV Transformer Bus,110.0,n,None,True
3,MV Transformer Bus,20.0,n,None,True
4,MV Main Bus,20.0,b,None,True
5,MV Bus 1,20.0,b,None,True
6,MV Bus 2,20.0,b,None,True


Alle create-Funktionen geben den pandapower-Index der erzeugten Komponente zurück. Variable bus6 entspricht damit dem Index des buses mit dem Namen "MV Bus 1" (welcher den Index 5 hat):


In [4]:
bus6

5

Wir nutzen diese Variablen, um weitere bus- und branch-Elemente hinzuzufügen.

### External Grid

<img src="pics/example_network_simple_ext_grid.png">

Jetzt wird eine External Grid-Verbindung erstellt, welche als slack-Knoten für die Lastflussberechnung fungiert. Die Spannung der External-Grid-Komponente wird auf 1.02 per unit gesetzt. Des Weiteren definieren wir einen Phasenwinkel von 50 Grad:

In [4]:
pp.create_ext_grid(net, bus1, vm_pu=1.02, va_degree=50) # Create an external grid connection

net.ext_grid #show external grid table

,name,bus,vm_pu,va_degree,in_service
0,None,0,1.02,50.0,True


### Transformatoren 

<img src="pics/example_network_simple_trafo.png">

Der Transformator verbindet die Hochspannungs- mit der Mittelspannungsseite des Netzwerks. Der Hochspannungsbus des Transformators wird mit Bus3 verbunden, während der Mittelspannungsbus mit Bus4 verbunden ist. Wir wählen einen Transformatortyp aus der in pandapower enthaltenen Standardtypenbibliothek aus: 25 MVA 110/20 kV.


In [5]:
trafo1 = pp.create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

Die detaillierten Trafo-Parameter, wie z.B. die Kurzschlussspannung, Nennleistung oder Eisenverluste, werden automatisch aus der Bibliothek geladen und in der Trafo-Tabelle gespeichert: 


In [6]:
net.trafo #show transformer table

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,110kV/20kV transformer,25 MVA 110/20 kV,2,3,25.0,110.0,20.0,12.0,0.41,14.0,...,0,-9,9,1.5,0.0,0,False,1,1.0,True


### Verbindungen

Das Netzwerk beinhaltet drei Mittelspannungsverbindungen und eine Hochspannungsverbindung. Die Bus-Anbindungen und Kabellängen sind im Diagramm angegeben:

<img src="pics/example_network_simple_lines.png">

Die Verbindungsparameter werden erneut aus der Standardbibliothek entnommen. Wir nutzen unterschiedliche Kabellängen und Standardtypen für jedes Kabel:

In [7]:
line1 = pp.create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV",  name="Line 1")
line2 = pp.create_line(net, bus5, bus6, length_km=2.0, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3 = pp.create_line(net, bus6, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4 = pp.create_line(net, bus7, bus5, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

Anschließend sieht die Tabelle wie folgt aus:

In [8]:
net.line # show line table

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line 1,N2XS(FL)2Y 1x300 RM/35 64/110 kV,0,1,10.0,0.0600,0.144,144.0,0.0,0.588,1.0,1,cs,True
1,Line 2,NA2XS2Y 1x240 RM/25 12/20 kV,4,5,2.0,0.1220,0.112,304.0,0.0,0.421,1.0,1,cs,True
2,Line 3,48-AL1/8-ST1A 20.0,5,6,3.5,0.5939,0.372,9.5,0.0,0.210,1.0,1,ol,True
3,Line 4,NA2XS2Y 1x240 RM/25 12/20 kV,6,4,2.5,0.1220,0.112,304.0,0.0,0.421,1.0,1,cs,True


### Schalter

Es gibt zwei Schalter, welche den Stromkreis unterbrechen können (je einen auf Hoch- und Mittelspannungsebene). Die Schalter sind zwischen Bus4 und Bus5 bzw. zwischen Bus2 und Bus3 platziert. Ein zwischen zwei Bussen eingesetzter Schalter wird mit dem Parameter et="b" definiert.

<img src="pics/example_network_simple_switches.png">

In [9]:
sw1 = pp.create_switch(net, bus2, bus3, et="b", type="CB", closed=True)
sw2 = pp.create_switch(net, bus4, bus5, et="b", type="CB", closed=True)

Weiterhin werden alle Bus/Line-Verbindungen im Mittelspannungsnetz mit load-break-Schaltern versehen ("LBS"). Diese Art Schalter wird mit dem Parameter et="l" versehen.


In [10]:
sw3 = pp.create_switch(net, bus5, line2, et="l", type="LBS", closed=True)
sw4 = pp.create_switch(net, bus6, line2, et="l", type="LBS", closed=True)
sw5 = pp.create_switch(net, bus6, line3, et="l", type="LBS", closed=True)
sw6 = pp.create_switch(net, bus7, line3, et="l", type="LBS", closed=False)
sw7 = pp.create_switch(net, bus7, line4, et="l", type="LBS", closed=True)
sw8 = pp.create_switch(net, bus5, line4, et="l", type="LBS", closed=True)

Die Tabelle der Schalter enthält nun alle definierten Komponenten. Die Spalte "bus" enthält den Index des Busses, an den der Schalter angeschlossen ist. Für Schalter des b-Typs enthält die Tabelle den Index des zweiten Buses. Im Falle des l-Typs enthält die Tabelle den Index der Verbindung, an welche der Schalter angeschlossen ist. Alle Schalter sind geschlossen.


In [11]:
net.switch # show switch table

,bus,element,et,type,closed,name,z_ohm
0,1,2,b,CB,True,None,0.0
1,3,4,b,CB,True,None,0.0
2,4,1,l,LBS,True,None,0.0
3,5,1,l,LBS,True,None,0.0
4,5,2,l,LBS,True,None,0.0
5,6,2,l,LBS,False,None,0.0
6,6,3,l,LBS,True,None,0.0
7,4,3,l,LBS,True,None,0.0


### Lasten

<img src="pics/example_network_simple_load.png">

Das Last-Element wird genutzt um aktive und reaktve Verbraucher zu modellieren. Es wird eine 2 MW / 3MVar-Last mit einem Skalierungsfaktor von 0.6 erzeugt.


In [12]:
pp.create_load(net, bus7, p_mw=2, q_mvar=4, scaling=0.6, name="load")

net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,load,6,2.0,4.0,0.0,0.0,NaN,0.6,True,wye


#### Spannungsabhängige Lasten - ZIP-Lastmodell

Man kann die Parameter `const_z_percent` und `const_i_percent` beobachten, die standardmäßig zu 0 gesetzt sind. Sie können eingesetzt werden, um einen spannungsabhängigen Lastanteil zu definieren. Dazu wird ein sogenanntes **ZIP load**-Modell verwendet, welches die Last als Kombination aus konstanter Leistung, konstantem Strom und konstanter Impedanz beschreibt.

Exemplarisch wird eine 2 MW/4 MVar-Last mit 30% konstanter Impedanz und 20% konstantem Strom erzeugt:

In [13]:
pp.create_load(net, bus7, p_mw=2, q_mvar=4, const_z_percent=30, const_i_percent=20, name="zip_load")

net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,load,6,2.0,4.0,0.0,0.0,NaN,0.6,True,wye
1,zip_load,6,2.0,4.0,30.0,20.0,NaN,1.0,True,wye


### Statischer Generator

<img src="pics/example_network_simple_sgen.png">

Der statische Generator wird genutzt, um aktive und reaktive Generatoren mit konstanter Leistung zu modellieren. Weil das Vorzeichensystem von pandapower immer die Sicht des Verbrauchers beschreibt, ist die aktive Leistung negativ, wenn Leistung erzeugt wird. Im Beispiel wird ein statischer Generator mit einer Leistung von 2 MW / 500 kVar genutzt.


In [14]:
pp.create_sgen(net, bus7, p_mw=2, q_mvar=-0.5, name="static generator")

net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source
0,static generator,6,2.0,-0.5,NaN,1.0,True,wye,True


### Spannungsgeregelter Generator

<img src="pics/example_network_simple_gen.png">

Die Generator-Komponente wird genutzt, um die konstante, spannungsgeregelte Generierung von Leistung zu modellieren. Der Generator wird mit einem Arbeitspunkt für die Spannung definiert:


In [15]:
pp.create_gen(net, bus6, p_mw=6, max_q_mvar=3, min_q_mvar=-3, vm_pu=1.03, name="generator") 

net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,type
0,generator,5,6.0,1.03,NaN,-3.0,3.0,1.0,False,True,None


### Shunt


<img src="pics/example_network_simple_shunt.png">

Ein shunt wird über dessen aktiven und reaktiven Verbrauch bei der Nennleistung charakterisiert. Zur Erinnerung: Das Vorzeichensystem basiert auf der Verbrauchersicht. Es wird eine Kapazitätsbank modelliert, weshalb die definierte reaktive Last negativ ist:
 

In [16]:
pp.create_shunt(net, bus3, q_mvar=-0.96, p_mw=0, name='Shunt')

net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,in_service
0,2,Shunt,-0.96,0.0,110.0,1,1,True


Die Berechnung kann jetzt gestartet werden:

In [ ]:
pp.runpp(net)

Nach der Berechnung werden dem Netz-Container Ergebnistabellen hinzugefügt, die wie folgt abgerufen werden können:

In [ ]:
net.res_line

### Lessons learned

Nach dem Durcharbeiten des Tutorials sollten Sie wissen:
    
- welche Schritte vor einer Berechnung durchzuführen sind
- welche Komponenten von pandapower zur Verfügung gestellt werden
- dass es eine Komponentenbibliothek in pandapower gibt
- Mit welchem Befehl eine Berechnung gestartet wird
- Wie man Ergebnisse auswertet